# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

geoapify_key


'76035bcb2a4f478f9e767751ba8e92a5'

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,palikir - national government center,6.9248,158.1611,25.41,94,100,1.03,FM,1718290900
1,1,port blair,11.6667,92.7500,29.02,84,40,4.63,IN,1718290900
2,2,kapa'a,22.0752,-159.3190,21.77,77,1,4.35,US,1718290900
3,3,santa luzia,-19.7697,-43.8514,23.76,51,0,3.60,BR,1718290900
4,4,fortuna,40.5982,-124.1573,11.19,91,100,3.09,US,1718290636


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5
)

# Display the map
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values

criteria = (
    (city_data_df['Max Temp'] < 27) & 
    (city_data_df['Max Temp'] > 21) & 
    (city_data_df['Wind Speed'] < 4.5) & 
    (city_data_df['Cloudiness'] == 0)
)
narrowed_city_data_df = city_data_df[criteria]

# Drop any rows with null values
cleaned_city_data = narrowed_city_data_df.dropna()


# Display sample data
cleaned_city_data

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,3,santa luzia,-19.7697,-43.8514,23.76,51,0,3.60,BR,1718290900
12,12,namsos,64.4662,11.4957,21.95,37,0,3.09,NO,1718290901
58,58,wailua homesteads,22.0669,-159.3780,21.03,77,0,3.97,US,1718290905
124,124,kambove,-10.8764,26.5969,22.95,37,0,2.07,CD,1718290911
129,129,newman,37.3138,-121.0208,21.24,73,0,1.79,US,1718290911
180,180,namibe,-15.1961,12.1522,24.78,52,0,3.98,AO,1718290917
200,200,winnemucca,40.9730,-117.7357,22.03,24,0,0.00,US,1718290918
218,218,kolonia,50.9333,6.9500,21.44,42,0,2.06,DE,1718290753
224,224,ilhabela,-23.7781,-45.3581,26.14,45,0,1.24,BR,1718290921
269,269,kuvandyk,51.4781,57.3552,26.22,62,0,2.90,RU,1718290926


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity

hotel_df = cleaned_city_data[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
3,santa luzia,BR,-19.7697,-43.8514,51,
12,namsos,NO,64.4662,11.4957,37,
58,wailua homesteads,US,22.0669,-159.3780,77,
124,kambove,CD,-10.8764,26.5969,37,
129,newman,US,37.3138,-121.0208,73,
180,namibe,AO,-15.1961,12.1522,52,
200,winnemucca,US,40.9730,-117.7357,24,
218,kolonia,DE,50.9333,6.9500,42,
224,ilhabela,BR,-23.7781,-45.3581,45,
269,kuvandyk,RU,51.4781,57.3552,62,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [13]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey" : geoapify_key,
    "limit" : 20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places?"

   

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    
    # Convert the API response to JSON format
    name_address = name_address.json()

  
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df



Starting hotel search
santa luzia - nearest hotel: Alcoboca
namsos - nearest hotel: Tinos hotell
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
kambove - nearest hotel: No hotel found
newman - nearest hotel: No hotel found
namibe - nearest hotel: Nelsal Pensao
winnemucca - nearest hotel: The Overland Hotel
kolonia - nearest hotel: Wasserturm Hotel Cologne
ilhabela - nearest hotel: Vila Kebaya
kuvandyk - nearest hotel: Отдыхающая локомотивных бригад
pollock pines - nearest hotel: Best Western Stagecoach Inn
kimberley - nearest hotel: Royal Paume Hotel
broome - nearest hotel: No hotel found
kawasaki - nearest hotel: フレックステイイン
sombrerete - nearest hotel: No hotel found
benguela - nearest hotel: Hotel Moibela
boa esperanca - nearest hotel: Hotel do Lago
iguape - nearest hotel: Pousada dos Lampiões


,City,Country,Lat,Lng,Humidity,Hotel Name
3,santa luzia,BR,-19.7697,-43.8514,51,Alcoboca
12,namsos,NO,64.4662,11.4957,37,Tinos hotell
58,wailua homesteads,US,22.0669,-159.3780,77,Hilton Garden Inn Kauai Wailua Bay
124,kambove,CD,-10.8764,26.5969,37,No hotel found
129,newman,US,37.3138,-121.0208,73,No hotel found
180,namibe,AO,-15.1961,12.1522,52,Nelsal Pensao
200,winnemucca,US,40.9730,-117.7357,24,The Overland Hotel
218,kolonia,DE,50.9333,6.9500,42,Wasserturm Hotel Cologne
224,ilhabela,BR,-23.7781,-45.3581,45,Vila Kebaya
269,kuvandyk,RU,51.4781,57.3552,62,Отдыхающая локомотивных бригад


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [16]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 1
)

# Display the map
hotel_map



:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)